## 기본 langchain UnstructuredPowerPointLoader 사용


In [3]:
from langchain_community.document_loaders import UnstructuredPowerPointLoader
from dotenv import load_dotenv
from pathlib import Path
from langchain_chroma import Chroma
from langchain_community.embeddings import OllamaEmbeddings
import os
import sys

from agent.config import VECTORSTORE_DIR

load_dotenv()

# Chroma 벡터스토어 생성
def get_vectorstore():
    collection_name = os.getenv('BASIC_PROPOSAL_DB_NAME')
    vectorstore = Chroma(
        persist_directory=VECTORSTORE_DIR['db'],
        embedding_function=OllamaEmbeddings(model="nomic-embed-text"),
        collection_name=collection_name
    )
    return vectorstore

# 벡터 스토어에 파일이 존재하는지 확인
def check_file_in_vectorstore(db, file_name):
    result = db.get()

    # 메타데이터가 있는지 확인
    if not result['metadatas']:
        return False
    
    found = False
    for metadata in result['metadatas']:
        if 'source' in metadata and file_name in metadata['source']:
            found = True
            break
    if found:
        print(f"{file_name} 파일이 이미 존재합니다.")
    else:
        print(f"{file_name} 파일이 존재하지 않습니다.")
        
    return found

# PPT 파일을 로드하여 vectorstore에 저장
def add_ppt_to_vectorstore(db, ppt_file_path):
    loader = UnstructuredPowerPointLoader(ppt_file_path, mode="elements")
    documents = loader.load_and_split()
    db.add_documents(documents)

# vectorstore 디렉토리가 존재하는지 확인하고 없으면 생성
def ensure_vectorstore_directory():
    persist_dir = Path(VECTORSTORE_DIR['db'])
    if not persist_dir.exists():
        persist_dir.mkdir(parents=True)
        print(f"벡터스토어 디렉토리 생성됨: {persist_dir}")
    else:
        print(f"벡터스토어 디렉토리가 이미 존재함: {persist_dir}")

# PPT 파일을 찾아서 목록 생성
def get_pptx_files():
    # documents 디렉토리 경로
    doc_dir = Path(VECTORSTORE_DIR['documents'])
    
    # .pptx 파일만 찾아서 목록 생성
    pptx_files = list(doc_dir.glob('*.pptx'))
    
    if not pptx_files:
        print("PPT 파일이 존재하지 않습니다.")
        return []
    
    print(f"발견된 PPT 파일 목록:")
    for file in pptx_files:
        print(f"- {file.name}")
    
    return pptx_files

In [5]:
ensure_vectorstore_directory()
db = get_vectorstore()
print(db.get())

try:
    pptx_files = get_pptx_files()
except Exception as e:
    print(f"PPT 파일을 찾는 중 오류 발생: {e}")


if not pptx_files:
    sys.exit(0)
else:
    for pptx in pptx_files:
        if check_file_in_vectorstore(db, pptx.name):
            print(f"{pptx.name} 파일이 이미 존재합니다.")
        else:
            try:    
                pptx_path = os.path.join(VECTORSTORE_DIR['documents'], pptx.name)    
                print(f"{pptx_path} 파일을 벡터스토어에 저장합니다.")
                add_ppt_to_vectorstore(db, pptx_path)
            except Exception as e:
                print(f"PPT 파일을 벡터스토어에 저장하는 중 오류 발생: {e}")



벡터스토어 디렉토리가 이미 존재함: C:\Users\iblue\Projects\proposal-agent\data\vectorstore
{'ids': [], 'embeddings': None, 'documents': [], 'uris': None, 'data': None, 'metadatas': [], 'included': [<IncludeEnum.documents: 'documents'>, <IncludeEnum.metadatas: 'metadatas'>]}
PPT 파일이 존재하지 않습니다.


SystemExit: 0